## Probe Match

In [1]:
%pwd

'/home/junyuchen/Lab/Probe-Match'

### Prokka - Contigs annotation

In [ ]:
prokka --outdir mydir --prefix mygenome contigs.fa

In [ ]:
prokka --outdir Root11 --prefix Root11 Root11.fna

```shell
for fasta in `ls | grep .fna`
do
    prokka --outdir /home/junyuchen/Lab/Probe-Match/Root-WGS-Annotated/${fasta:0:-4} --prefix ${fasta:0:-4} $fasta
done
```

Parising GFF file

### Extract rRNA

#### bcbio-gff
`conda install -c bioconda bcbio-gff`

In [1]:
from BCBio import GFF

In [ ]:
pip install bcbio-gff

In [2]:
%pwd

'/home/junyuchen/Lab/Probe-Match'

```python
from BCBio import GFF

in_file = "your_file.gff"

limit_info = dict(
        gff_id = ["chr1"],
        gff_source = ["Coding_transcript"])

in_handle = open(in_file)
for rec in GFF.parse(in_handle, limit_info=limit_info):
    print rec.features[0]
in_handle.close()
```

`product=23S ribosomal RNA`   
`product=16S ribosomal RNA`

In [2]:
from BCBio import GFF

In [3]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [4]:
in_file = "./demo/data/Root123D2.gff"

In [5]:
import pprint
from BCBio.GFF import GFFExaminer

#in_file = "your_file.gff"
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

{'gff_id': {('contig_1',): 1896,
            ('contig_2',): 1211,
            ('contig_3',): 896,
            ('contig_4',): 371,
            ('contig_5',): 222,
            ('contig_6',): 53,
            ('contig_7',): 5,
            ('contig_8',): 4},
 'gff_source': {('Aragorn:1.2',): 55,
                ('Prodigal:2.6',): 4600,
                ('barrnap:0.9',): 3},
 'gff_source_type': {('Aragorn:1.2', 'tRNA'): 54,
                     ('Aragorn:1.2', 'tmRNA'): 1,
                     ('Prodigal:2.6', 'CDS'): 4600,
                     ('barrnap:0.9', 'rRNA'): 3},
 'gff_type': {('CDS',): 4600, ('rRNA',): 3, ('tRNA',): 54, ('tmRNA',): 1}}


```shell
in_handle = open(in_file)
for rec in GFF.parse(in_handle):
    print(rec)
in_handle.close()
```

In [9]:
#in_file = "/home/junyuchen/Lab/Probe-Match/Root-WGS/Root11/Root11.gff"
limit_info = dict(gff_type = ["rRNA"], gff_source = ["barrnap:0.9"])
in_handle = open(in_file)
for record in GFF.parse(in_handle, limit_info=limit_info):
    #print(type(rec.features))
    #print(rec.features[0].qualifiers["product"])
    #print(rec.id)
    #print(rec.features[0])
    print("ID = %s, length %i, with %i features"
          % ( record.id, len(record.seq), len(record.features)))
    print (record.features)
    
in_handle.close()

ID = contig_1, length 1914680, with 0 features
[]
ID = contig_2, length 1264344, with 0 features
[]
ID = contig_3, length 926414, with 3 features
[SeqFeature(FeatureLocation(ExactPosition(67862), ExactPosition(67971), strand=-1), type='rRNA', id='MOFEJJPE_03177'), SeqFeature(FeatureLocation(ExactPosition(68060), ExactPosition(70869), strand=-1), type='rRNA', id='MOFEJJPE_03178'), SeqFeature(FeatureLocation(ExactPosition(71675), ExactPosition(73160), strand=-1), type='rRNA', id='MOFEJJPE_03181')]
ID = contig_4, length 409661, with 0 features
[]
ID = contig_5, length 209741, with 0 features
[]
ID = contig_6, length 59929, with 0 features
[]
ID = contig_7, length 14163, with 0 features
[]
ID = contig_8, length 5484, with 0 features
[]


In [15]:
import os

In [19]:
ouputDir = "/home/junyuchen/Lab/Probe-Match/Probe-Match/demo/result"

In [26]:
limit_info = dict(gff_type = ["rRNA"])
        
def FindrRNA(path, tempFile):
    
    in_handle = open(path)
    for record in GFF.parse(in_handle, limit_info=limit_info):
        if len(record.features) != 0:
            for i in range(len(record.features)):
                if '23S ribosomal RNA' in record.features[i].qualifiers["product"][0]:
                    seq_23S = SeqRecord(record.features[i].location.extract(record.seq),\
                                    id=tempFile[:-4],\
                                    description=str(record.id +'-'+record.features[i].id+'-'+ record.features[i].qualifiers["product"][0]+'-'+str(len(record.seq))))
                    SeqIO.write(seq_23S, os.path.join(ouputDir, tempFile[:-4]+"_23S_rRNA.fasta"), "fasta")
                elif '16S ribosomal RNA' in record.features[i].qualifiers["product"][0]:
                    seq_16S = SeqRecord(record.features[i].location.extract(record.seq),\
                                    id=tempFile[:-4],\
                                    description=str(record.id +'-'+record.features[i].id+'-'+ record.features[i].qualifiers["product"][0]+'-'+str(len(record.seq))))
                    SeqIO.write(seq_16S, os.path.join(ouputDir, tempFile[:-4]+"_16S_rRNA.fasta"), "fasta")

    in_handle.close()

In [27]:
FindrRNA("/home/junyuchen/Lab/Probe-Match/Probe-Match/demo/data/Root123D2.gff", "Root123D2.gff")

In [ ]:
python /home/junyuchen/Lab/Probe-Match/Probe-Match/Scripts/Extract-rRNA.py -i /home/junyuchen/Lab/Probe-Match/Probe-Match/demo/data -o /home/junyuchen/Lab/Probe-Match/Probe-Match/demo/result